# 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助你：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(agent101)`，即可执行


In [1]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate agent101

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "agent101" ]; then
    echo "✅ 当前单元格已成功激活到 agent101 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(agent101)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate agent101"
fi

echo "========================================="


== Conda 环境检查报告 (仅针对当前 Bash 子进��) ==
✅ 当前单元格已成功激活到 agent101 环境。
✅ 正在使用的环境路径: /root/miniconda3/envs/agent101

💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。
   如果需要后续单元格也使用此环境，请执行以下操作:
   1. 检查 Notebook 右���角是否已选择 'python(agent101)'。


In [2]:
# 2. 设置pip 为清华源
%pip config list -v set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config list -v


Writing to /root/.config/pip/pip.conf
Note: you may need to restart the kernel to use updated packages.
ERROR: Need an action (debug, edit, get, list, set, unset) to perform.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT


env: HF_ENDPOINT=https://hf-mirror.com
https://hf-mirror.com


In [4]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助你：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                               |
|:-------------|:-----------------------------------------------------------------------------------|
| 操作系统     | Linux Ubuntu 22.04.4 LTS                                                           |
| CPU 信息     | 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz (1 physical cores, 2 logical cores) |
| 内存信息     | 3.78 GB (Available: 1.65 GB)                                                       |
| GPU 信息     | No GPU found (nvidia-smi not found)                                                |
| CUDA 信息    | CUDA not found                                                                     |
| Python 版本  | 3.10.18                                                                            |
| Conda 版本   | conda 24.4.0                                         

# 第五节：格式化输出和为大模型预设响应

## 设置

运行以下设置单元格来加载你的API密钥并建立`get_completion`辅助函数。


In [5]:
# 安装OpenAI库
%pip install openai==1.107.0

# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    # 新增了prefill参数用于预填充文本，默认值为空字符串
    def get_completion(prompt: str, system_prompt="", prefill=""):
        """
        获取大模型的完成响应
        
        参数:
            prompt (str): 用户提示内容
            system_prompt (str): 系统提示（可选）
            prefill (str): 预填充文本（可选），用于引导大模型的响应开始
        
        返回:
            str: 大模型的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 如果有预填充内容，添加助手消息
        if prefill:
            messages.append({"role": "assistant", "content": prefill})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 使用的模型名称
            messages=messages,             # 消息列表
            max_tokens=2000,              # 最大生成的token数量
            temperature=0.0               # 温度参数，0表示更确定性的回答
        )
        
        # 如果有预填充内容，将其与生成的内容组合
        if prefill:
            return prefill + response.choices[0].message.content
        else:
            return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://api.apiyi.com/v1
  🤖 模型: deepseek-chat
  🌐 API地址: https://api.apiyi.com/v1
  🔑 API密钥: sk-AGnnT...A14f

✅ 使用统一配置管理成功！


---

## 课程

**大模型可以以多种方式格式化其输出**。你只需要明确要求它这样做！

其中一种方式是使用XML标签将响应与其他多余文本分离。你已经学会了可以使用XML标签使你的提示对大模型更清晰、更易解析。事实证明，你还可以要求大模型**使用XML标签使其输出对人类更清晰、更容易理解**。

### 技术要点

1. **XML标签格式化**：指导大模型使用结构化的XML标签来组织输出
2. **预设响应（Prefill）**：通过在assistant角色中预填充部分内容来引导大模型的响应方向
3. **结构化输出**：要求大模型按照特定的格式（如JSON、表格、列表等）输出内容
4. **多段落组织**：使用标记来分离不同类型的信息


### 示例 1：使用XML标签格式化输出

要求大模型使用XML标签来组织其回答，使输出更加结构化和易于解析。


In [6]:
# 示例1：XML标签格式化输出
prompt = """
请分析人工智能对教育行业的影响，并使用以下XML标签格式化你的回答：

<分析>
在这里提供主要分析内容
</分析>

<优势>
列出AI在教育中的优势
</优势>

<挑战>
列出AI在教育中面临的挑战
</挑战>

<建议>
提供实施建议
</建议>
"""

response = get_completion(prompt)
print("🤖 大模型回答:")
print(response)


🤖 GPT回答:
<分析>
人工智能（AI）正在深刻改变教育行业，从教学方式到学习体验都带来了革新。AI技术通过个性化学习、自动化管理和智能评估等方式，提升了教育效率和质量。同时，AI也面临数据隐私、技术普及和教师角色转变等挑战。未来，AI与教育的深度融合需要平衡技术创新与人文关怀。
</分析>

<优势>
1. **个性化学习**：AI可以根据学生的学习进度和能力提供定制化内容，提高学习效率。
2. **自动化管理**：AI可以处理考勤、作业批改等重复性任务，减轻教师负担。
3. **智能辅导**：AI驱动的聊天机器人或虚拟助手可以24/7为学生答疑解惑。
4. **数据分析**：AI能分析学生表现数据，帮助教师发现学习难点并调整教学策略。
5. **教育资源普及**：AI可以通过在线平台将优质教育资源扩展到偏远地区。
</优势>

<挑战>
1. **数据隐私问题**：AI需要大量学生数据，可能引发隐私泄露风险。
2. **技术鸿沟**：并非所有学校或地区都能负担AI技术的成本，可能加剧教育不平等。
3. **教师适应性**：部分教师可能对AI技术不熟悉，需要额外培训。
4. **情感缺失**：AI难以替代教师在情感支持和人际互动中的作用。
5. **伦理问题**：AI算法的偏见可能影响教育公平性。
</挑战>

<建议>
1. **分阶段实施**：从试点项目开始，逐步推广AI技术，避免盲目投入。
2. **加强数据保护**：制定严格的数据隐私政策，确保学生信息安全。
3. **教师培训**：为教师提供AI工具使用培训，帮助其适应新技术。
4. **人机协作**：明确AI的辅助定位，保留教师在情感教育和创新教学中的核心作用。
5. **政策支持**：政府应提供资金和政策支持，缩小技术普及差距。
</建议>


In [7]:
# 示例2：使用预设响应引导大模型风格
prompt = """
作为一名资深的Python开发专家，请解释什么是装饰器，并提供一个实用的示例。
"""

# 使用预设响应来引导大模型以特定的风格和格式回答
prefill = """## 🐍 Python装饰器详解

### 概念理解
装饰器是Python中的一个重要概念，"""

response = get_completion(prompt, prefill=prefill)
print("🤖 使用预设响应的回答:")
print(response)
print("\n" + "="*60 + "\n")

# 对比：不使用预设响应的回答
response_normal = get_completion(prompt)
print("🤖 普通回答:")
print(response_normal)


🤖 使用预设响应的回答:
## 🐍 Python装饰器详解

### 概念理解
装饰器是Python中的一个重要概念，## 🐍 Python装饰器详解  

### **1. 装饰器概念**  
装饰器（Decorator）是Python中的一种高级语法，**用于动态修改或扩展函数或类的行为**，而不需要修改其源代码。装饰器本质上是一个**可调用对象**（函数或类），它接受一个函数作为参数，并返回一个新的函数。  

装饰器的核心特点：  
- **不改变原函数的代码逻辑**  
- **增强函数的功能**（如日志记录、性能测试、权限校验等）  
- **遵循“开放-封闭”原则**（对扩展开放，对修改封闭）  

---

### **2. 基本语法**  
装饰器使用 `@` 符号，放在函数定义之前。  

#### **示例1：最简单的装饰器**  
```python
def my_decorator(func):
    def wrapper():
        print("函数执行前...")
        func()  # 调用原函数
        print("函数执行后...")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()
```
**输出：**  
```
函数执行前...
Hello!
函数执行后...
```
✅ **解析：**  
- `my_decorator` 是一个装饰器，它接收 `say_hello` 函数作为参数。  
- `wrapper` 是内部函数，负责在调用原函数前后添加额外逻辑。  
- `@my_decorator` 相当于 `say_hello = my_decorator(say_hello)`。  

---

### **3. 带参数的装饰器**  
如果被装饰的函数有参数，需要在 `wrapper` 中使用 `*args` 和 `**kwargs` 来接收任意参数。  

#### **示例2：计算函数执行时间**  
```python
import time

def timing_decorator(func):
    def wrapper(*args, **kwar

### 示例 2：JSON格式输出

要求大模型输出标准的JSON格式，便于程序解析和处理。


In [8]:
# 示例3：JSON格式输出
prompt = """
请为一个在线书店设计一个图书数据结构，包含以下信息：
- 书名
- 作者
- ISBN
- 价格
- 分类
- 出版日期
- 库存数量
- 评分

请以标准JSON格式输出，并提供3本示例图书的数据。

输出格式要求：
1. 使用中文字段名
2. 价格用数字类型
3. 评分范围0-5
4. 出版日期格式：YYYY-MM-DD
"""

response = get_completion(prompt)
print("🤖 JSON格式输出:")
print(response)

🤖 JSON格式输出:
```json
[
  {
    "书名": "三体",
    "作者": "刘慈欣",
    "ISBN": "9787536692930",
    "价格": 23.9,
    "分类": "科幻",
    "出版日期": "2008-01-01",
    "库存数量": 150,
    "评分": 4.8
  },
  {
    "书名": "活着",
    "作者": "余华",
    "ISBN": "9787506365437",
    "价格": 28.0,
    "分类": "文学",
    "出版日期": "2012-08-01",
    "库存数量": 200,
    "评分": 4.9
  },
  {
    "书名": "Python编程：从入门到实践",
    "作者": "Eric Matthes",
    "ISBN": "9787115428028",
    "价格": 89.0,
    "分类": "计算机",
    "出版日期": "2016-07-01",
    "库存数量": 75,
    "评分": 4.7
  }
]
```


In [9]:
# 实验1：自定义XML标签格式
# 在这里尝试你自己的格式化提示词

custom_prompt = """
# 请在这里输入你的提示词，尝试使用自定义的XML标签格式
# 例如：分析一个技术趋势、评估一个产品、制定一个计划等
"""

# 取消下面的注释来运行你的实验
# response = get_completion(custom_prompt)
# print("🔬 你的实验结果:")
# print(response)

print("💡 提示：修改上面的 custom_prompt 变量，然后取消注释运行代码")


💡 提示：修改上面的 custom_prompt 变量，然后取消注释运行代码


In [10]:
# 实验2：预设响应风格测试
# 选择一个你感兴趣的主题
your_prompt = "请解释区块链技术的工作原理"

# 尝试不同的预设响应风格
prefill_styles = {
    "学术风格": "# 区块链技术原理分析\n\n## 摘要\n\n区块链作为一种分布式账本技术，",
    "科普风格": "👋 大家好！今天我们来聊聊区块链这个神奇的技术！\n\n想象一下，",
    "技术文档风格": "## 区块链技术文档\n\n### 1. 概述\n\n区块链（Blockchain）是",
    "问答风格": "Q: 什么是区块链技术？\nA: 区块链是一种",
}

print("🎨 不同风格的预设响应效果对比：")
print("="*60)

for style_name, prefill in prefill_styles.items():
    print(f"\n📝 {style_name}:")
    print("-" * 30)
    response = get_completion(your_prompt, prefill=prefill)
    # 只显示前200个字符作为预览
    preview = response[:200] + "..." if len(response) > 200 else response
    print(preview)
    print()

# 你也可以尝试自己的预设响应
print("\n🔧 尝试你自己的预设响应:")
your_prefill = """
# 在这里输入你想要的预设响应开头
"""

# 取消注释来测试你的预设响应
# your_response = get_completion(your_prompt, prefill=your_prefill)
# print("🎯 你的预设响应效果:")
# print(your_response)


🎨 不同风格的预设响应效果对比：

📝 学术风格:
------------------------------
# 区块链技术原理分析

## 摘要

区块链作为一种分布式账本技术，# 区块链技术的工作原理详解

区块链是一种去中心化的分布式账本技术，其核心工作原理可以概括为以下几个关键方面：

## 1. 分布式网络结构

区块链网络由众多节点（计算机）组成，每个节点都保存着完整的账本副本。这种去中心化结构意味着：
- 没有单一控制点或故障点
- 所有参与者共同维护系统
- 新交易需要网络多数节点验证才能...


📝 科普风格:
------------------------------
👋 大家好！今天我们来聊聊区块链这个神奇的技术！

想象一下，👋 大家好！今天我们来聊聊区块链这个神奇的技术！  

想象一下，**一群人共同维护一本公开的“数字账本”**，账本里的每一页（区块）都记录着交易信息，而且每一页都通过特殊密码（哈希值）和前一页紧紧绑定，形成一条无法篡改的“链”——这就是区块链的核心思想！  

---

### 🌟 **区块链工作原理的通俗版解释**  
1️⃣ **...


📝 技术文档风格:
------------------------------
## 区块链技术文档

### 1. 概述

区块链（Blockchain）是### 1. 概述  
区块链（Blockchain）是一种**去中心化**的分布式账本技术，通过密码学、共识机制和点对点网络，实现数据不可篡改、透明可追溯的特性。其核心思想是：将交易数据按时间顺序打包成区块，并通过哈希值串联成链，由全网节点共同维护和验证，无需依赖单一中心化机构。  

---

### 2. 核心工作...


📝 问答风格:
------------------------------
Q: 什么是区块链技术？
A: 区块链是一种区块链是一种**去中心化、分布式账本技术**，通过密码学保证数据不可篡改，并实现无需第三方信任的可靠交易记录。以下是其核心工作原理的逐步解析：

---

### 1. **去中心化网络结构**
   - **节点网络**：区块链由多个节点（计算机）组成，每个节点独立存储完整的账本副本，没有中心化服务器。
   - **共识机制**：所有节点通过算法（如...




### 示例 3：复合格式输出

结合多种格式化技术，创建复杂的结构化输出。


In [11]:
# 实验3：复合格式输出
prompt = """
请为一个新的社交媒体应用制定产品规划，要求：

1. 使用XML标签组织内容结构
2. 在技术部分使用表格格式
3. 在时间规划部分使用JSON格式
4. 包含代码示例使用代码块格式

具体要求：
<产品概述>
简要描述产品定位和核心功能
</产品概述>

<目标用户>
用户画像分析
</目标用户>

<功能规划>
核心功能列表和优先级（用表格格式）
</功能规划>

<技术架构>
技术选型对比表格（包含前端、后端、数据库等）
</技术架构>

<开发计划>
6个月的开发时间线（JSON格式，包含阶段、任务、时间等字段）
</开发计划>

<API示例>
提供用户注册接口的代码示例
</API示例>
"""

# 使用预设响应确保格式正确
prefill = """📱 **社交媒体应用产品规划**

<产品概述>
本产品是一个面向年轻用户的"""

response = get_completion(prompt, prefill=prefill)
print("🔄 复合格式化输出:")
print(response)


🔄 复合格式化输出:
📱 **社交媒体应用产品规划**

<产品概述>
本产品是一个面向年轻用户的```xml
<社交媒体应用产品规划>

<产品概述>
  <定位>面向Z世代的沉浸式短视频社交平台</定位>
  <核心功能>
    - AI驱动的智能内容推荐
    - 多人在线协作创作工具
    - 元宇宙风格虚拟社交空间
    - 实时AR特效互动
  </核心功能>
</产品概述>

<目标用户>
  <用户画像>
    <年龄段>16-28岁</年龄段>
    <特征>
      - 日均社交APP使用时长>3小时
      - 偏好视觉化沟通方式
      - 愿意为虚拟商品付费
      - 关注亚文化圈层
    </特征>
  </用户画像>
</目标用户>

<功能规划>
  <table>
    <tr>
      <th>功能模块</th><th>优先级</th><th>MVP版本</th>
    </tr>
    <tr>
      <td>短视频创作工具</td><td>P0</td><td>✓</td>
    </tr>
    <tr>
      <td>AR实时滤镜</td><td>P0</td><td>✓</td>
    </tr>
    <tr>
      <td>虚拟直播间</td><td>P1</td><td>✗</td>
    </tr>
    <tr>
      <td>数字藏品商城</td><td>P2</td><td>✗</td>
    </tr>
  </table>
</功能规划>

<技术架构>
  <table>
    <tr>
      <th>技术栈</th><th>候选方案</th><th>最终选择</th><th>理由</th>
    </tr>
    <tr>
      <td>前端</td><td>React Native/Flutter</td><td>Flutter</td><td>跨平台一致性</td>
    </tr>
    <tr>
      <td>后端</td><td>Node.js/Go</td><td>Go</td><td>高并发处理</td>
    </tr>
    <tr>
      <td>数据库</td>

---

## 📚 总结与最佳实践

### 格式化输出的关键要点：

1. **明确性**：明确告诉大模型你期望的输出格式
2. **一致性**：使用一致的标签和结构来组织信息
3. **可解析性**：确保输出格式便于程序或人类解析
4. **适用性**：根据用途选择合适的格式（JSON、XML、表格等）

### 预设响应的最佳实践：

1. **风格引导**：通过预设响应建立特定的回答风格
2. **格式控制**：确保输出格式的一致性
3. **专业角色**：让大模型扮演特定的专业角色
4. **长度控制**：预设响应应该简洁但足够引导方向

### 实际应用场景：

- 📊 **数据分析报告**：使用XML标签组织分析结果
- 🔧 **技术文档**：结合代码块和表格展示技术方案
- 📋 **项目计划**：使用JSON格式输出结构化的项目信息
- 🎯 **产品评估**：通过预设响应确保评估的专业性和一致性

通过掌握这些格式化技术，你可以让大模型输出更加结构化、专业化的内容，大大提升工作效率！
